In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications.mobilenet import preprocess_input as mobilenet_preprocess

# ——— your folders ———
TRAIN_DIR = r'D:\iot project\train'
VAL_DIR   = r'D:\iot project\val'
IMG_SIZE  = (224, 224)
BATCH     = 32
NUM_CLASSES = 2  # cow,hen

# 1. Datasets
train_ds = image_dataset_from_directory(
    TRAIN_DIR,
    label_mode='int',
    image_size=IMG_SIZE,
    batch_size=BATCH,
    shuffle=True,
    seed=42
).map(lambda x, y: (mobilenet_preprocess(x), y)) \
 .prefetch(tf.data.AUTOTUNE)

val_ds = image_dataset_from_directory(
    VAL_DIR,
    label_mode='int',
    image_size=IMG_SIZE,
    batch_size=BATCH,
    shuffle=False
).map(lambda x, y: (mobilenet_preprocess(x), y)) \
 .prefetch(tf.data.AUTOTUNE)

# 2. Build MobileNetV1-based model
base = tf.keras.applications.MobileNet(
    input_shape=(*IMG_SIZE, 3),
    alpha=0.25,
    weights='imagenet',     # transfer learn from ImageNet
    include_top=False
)
base.trainable = False     # freeze for initial training

model = models.Sequential([
    layers.Input(shape=(*IMG_SIZE, 3)),
    base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASSES, activation='softmax'),
])

# 3. Compile & train head
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history1 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)

# 4. Unfreeze some of the base for fine-tuning
base.trainable = True
for layer in base.layers[:50]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history2 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

# 5. Optional: Combine histories or save model
model.save('mobilenetv1_finetuned.h5')
print("✅ Done — MobileNetV1 training finished.")


Found 320 files belonging to 2 classes.
Found 80 files belonging to 2 classes.
Epoch 1/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - accuracy: 0.6446 - loss: 0.7128 - val_accuracy: 0.9000 - val_loss: 0.2578
Epoch 2/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 472ms/step - accuracy: 0.8892 - loss: 0.2675 - val_accuracy: 0.9500 - val_loss: 0.1323
Epoch 3/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 499ms/step - accuracy: 0.9529 - loss: 0.1317 - val_accuracy: 0.9875 - val_loss: 0.0803
Epoch 4/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 391ms/step - accuracy: 0.9827 - loss: 0.0860 - val_accuracy: 0.9875 - val_loss: 0.0622
Epoch 5/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 428ms/step - accuracy: 0.9971 - loss: 0.0602 - val_accuracy: 0.9875 - val_loss: 0.0410
Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 43s 870ms/step - accuracy: 0.9322 - loss: 0.1617 - val_accuracy: 1.0000 - val_loss: 0.0255
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 601ms/step - accuracy: 0.9918 - loss: 0.0432 - val_accuracy: 1.0000 - val_loss: 0.0252
Epoch 3/10
10/10 ━━━━━━━━━━━━━━

✅ Done — MobileNetV1 training finished.


In [5]:
import tensorflow as tf
import numpy as np

model = tf.keras.models.load_model(r'D:\iot project\mobilenetv1_finetuned.h5', compile=False)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

def rep_data_gen():
    for _ in range(100):
        # random float in [0,1], matching your model's float32 input
        dummy = np.random.random_sample((1,224,224,3)).astype(np.float32)
        yield [dummy]

converter.representative_dataset = rep_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# Keep the I/O as float32 so it matches your model signature
# (the weights & activations will still be quantized to int8 internally)
converter.inference_input_type = tf.float32
converter.inference_output_type = tf.float32

tflite_model = converter.convert()
with open('mobilenetv1_quant.tflite', 'wb') as f:
    f.write(tflite_model)

print("✅ Quantization complete, saved mobilenetv1_quant.tflite")


INFO:tensorflow:Assets written to: C:\Users\white\AppData\Local\Temp\tmp4i7vo20j\assets


INFO:tensorflow:Assets written to: C:\Users\white\AppData\Local\Temp\tmp4i7vo20j\assets


Saved artifact at 'C:\Users\white\AppData\Local\Temp\tmp4i7vo20j'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_3')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  2488831474576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2488831473808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2488831475344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2488831474768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2488831474384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2488831474192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2488876843664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2488876844240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2488831473424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2488876843088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2488

c:\Users\white\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\lite\python\convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


✅ Quantization complete, saved mobilenetv1_quant.tflite


In [6]:
import tensorflow as tf
import numpy as np

import os

# ——— EDIT THESE PATHS IF NEEDED ———
TFLITE_MODEL_PATH = r'D:\iot project\mobilenetv1_quant.tflite'
VAL_DIR            = r'D:\iot project\val'     # cow/,goat/Hen
BATCH_SIZE         = 32
IMG_SIZE           = (224, 224)

# 1. Load validation dataset (values in [0,255], dtype uint8)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    VAL_DIR,
    labels='inferred',
    label_mode='int',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=False
)

# 2. Load TFLite interpreter
interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL_PATH)
interpreter.allocate_tensors()
input_details  = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

print("→ Model input details:", input_details)
print("→ Model expects dtype:", input_details['dtype'],
      "shape:", input_details['shape'])

# 3. Evaluate
total, correct = 0, 0

for batch_images, batch_labels in val_ds:
    # Convert to numpy for feeding the interpreter
    imgs = batch_images.numpy()           # shape (batch,128,128,3), uint8

    # 3a. Preprocess depending on expected input dtype
    if input_details['dtype'] == np.uint8:
        # Model wants uint8 [0..255]
        batch_input = imgs.astype(np.uint8)

    elif input_details['dtype'] == np.float32:
        # Model wants float32 in [0..1]
        batch_input = (imgs / 255.0).astype(np.float32)

    else:
        raise ValueError(f"Unsupported input dtype: {input_details['dtype']}")

    # 3b. Run per-image inference
    for img, true_label in zip(batch_input, batch_labels.numpy()):
        # Expand dims to match [1, h, w, c]
        interpreter.set_tensor(input_details['index'], np.expand_dims(img, 0))
        interpreter.invoke()
        output = interpreter.get_tensor(output_details['index'])[0]  # e.g. shape [3]
        pred_label = np.argmax(output)

        if pred_label == true_label:
            correct += 1
        total += 1

# 4. Report
acc = 100 * correct / total if total else 0
print(f'Validation accuracy: {acc:.2f}%  ({correct}/{total})')


Found 80 files belonging to 2 classes.
→ Model input details: {'name': 'serving_default_input_layer_3:0', 'index': 0, 'shape': array([  1, 224, 224,   3], dtype=int32), 'shape_signature': array([ -1, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
→ Model expects dtype: <class 'numpy.float32'> shape: [  1 224 224   3]


c:\Users\white\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


Validation accuracy: 98.75%  (79/80)
